In [1]:
%matplotlib inline
#
from lrgsglib.shared import *
from lrgsglib.core import *
from lrgsglib.Ccore.statsys.RBIsingM.storer import ising_model_store

In [2]:
move_to_rootf()

Current working directory: /home/opisthofulax/PhD/projects/complex-networks/lrgsglib


### Energy levels

In [3]:
side = 48
geo = 'squared'
pbc = True
pflip = 0.5
howmany = 100
cmap = restr_twilight

In [7]:
links = random.sample()

In [10]:
l.GraphReprDict['G'].get_edge_data(0, 1)

{'weight': 1}

In [14]:
nx.get_edge_attributes(l.GraphReprDict['G'], 'weight')

{(0, 48): 1,
 (0, 1): 1,
 (0, 2256): 1,
 (0, 47): 1,
 (1, 49): 1,
 (1, 2): 1,
 (1, 2257): 1,
 (2, 50): 1,
 (2, 3): 1,
 (2, 2258): 1,
 (3, 51): 1,
 (3, 4): 1,
 (3, 2259): 1,
 (4, 52): 1,
 (4, 5): 1,
 (4, 2260): 1,
 (5, 53): 1,
 (5, 6): 1,
 (5, 2261): 1,
 (6, 54): 1,
 (6, 7): 1,
 (6, 2262): 1,
 (7, 55): 1,
 (7, 8): 1,
 (7, 2263): 1,
 (8, 56): 1,
 (8, 9): 1,
 (8, 2264): 1,
 (9, 57): 1,
 (9, 10): 1,
 (9, 2265): 1,
 (10, 58): 1,
 (10, 11): 1,
 (10, 2266): 1,
 (11, 59): 1,
 (11, 12): 1,
 (11, 2267): 1,
 (12, 60): 1,
 (12, 13): 1,
 (12, 2268): 1,
 (13, 61): 1,
 (13, 14): 1,
 (13, 2269): 1,
 (14, 62): 1,
 (14, 15): 1,
 (14, 2270): 1,
 (15, 63): 1,
 (15, 16): 1,
 (15, 2271): 1,
 (16, 64): 1,
 (16, 17): 1,
 (16, 2272): 1,
 (17, 65): 1,
 (17, 18): 1,
 (17, 2273): 1,
 (18, 66): 1,
 (18, 19): 1,
 (18, 2274): 1,
 (19, 67): 1,
 (19, 20): 1,
 (19, 2275): 1,
 (20, 68): 1,
 (20, 21): 1,
 (20, 2276): 1,
 (21, 69): 1,
 (21, 22): 1,
 (21, 2277): 1,
 (22, 70): 1,
 (22, 23): 1,
 (22, 2278): 1,
 (23, 71): 1,


In [12]:
random_values = np.random.rand(l.N)
result = np.where(random_values < pflip, 1, -1)

In [13]:
result

array([-1,  1,  1, ..., -1,  1, -1])

In [8]:
l.flip_sel_edges(links)


TypeError: cannot unpack non-iterable numpy.int64 object

In [5]:
l = Lattice2D(side, geo, pbc=pbc, pflip=pflip)


In [4]:
l = Lattice2D(side, geo, pbc=pbc, pflip=pflip)
# np.random.seed(1)
# random.seed(1)
l.flip_sel_edges()
l.compute_k_eigvV(howmany=howmany, MODE_dynspec="scipy")
energies = np.array([-l.compute_rbim_energy_eigV(_) for _ in range(howmany)])
en0 = np.max(np.abs(energies))
energies /= l.N

TypeError: cannot unpack non-iterable numpy.int64 object

In [ ]:
fig, (ax, ax1) = plt.subplots(figsize=(10, 7), ncols=2, width_ratios=[1, 3])

ylabelPos = []
xlabelPos = []

y_min, y_max = min(energies), max(energies)
y_range = y_max - y_min
thresh = 0.05
threshold = y_range * thresh  # Adjust the fraction as needed

for i, en in enumerate(energies):
    color = cmap(i / len(energies)) if i != 0 else 'red'
    text_x = 1
    text_y = en
    if ylabelPos and any(abs(text_y - pos) < threshold for pos in ylabelPos):
        indices = [i for i, pos in enumerate(ylabelPos) 
                   if abs(text_y - pos) < threshold]
        index = max(indices, key=lambda i: xlabelPos[i])
        text_x = xlabelPos[index] +.3
    ax.axhline(en, 0.05, 1-0.05, lw=4, color=color, zorder=len(energies)-i+1)
    # ax.text(text_x+.05, text_y, fr'$E(|\lambda_{{{i}}}\rangle)$', 
    #          va='center', ha='left', fontsize=16, color=color)
    ylabelPos.append(text_y)
    xlabelPos.append(text_x)


# Customize the axes to remove the top and right box
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')


ax.spines['left'].set_position(('outward', 10))
ax.spines['bottom'].set_position(('outward', 10))
# # Add grid
# plt.ylim(-1, 1)
ax.grid(True)
fig.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(0, len(energies))), ax=ax)
x, pdf = powerlaw.pdf(np.abs(l.eigV[0]))
ax1.loglog(x[1:], pdf, 'k-', lw=2)
ax1.axvline(np.finfo(float).eps, color='red', lw=2)

In [ ]:
replicas = 100

In [ ]:
def compute_overlap(matrix1, matrix2):
    return np.dot(matrix1.flatten(), matrix2.flatten()) / matrix1.size
eigV = []
l = Lattice2D(side, geo, pbc=pbc, pflip=pflip)
for i in range(replicas):
    l.flip_random_fract_edges()
    l.compute_k_eigvV(howmany=1, MODE_dynspec="numpy")
    eigV.append(l.eigV[0].reshape(l.syshape))
# Compute all pairs of overlaps
overlaps = []
for i in range(replicas):
    for j in range(i + 1, replicas):
        q = compute_overlap(eigV[i], eigV[j])
        overlaps.append(q)

### C implementation (IsingDynamics `class`)

In [ ]:
side = 100
geo = 'squared'
pflip = 0.1
T_ising = 1
navg = 5
eqSTEP = 10
#
IMPORT = False

In [ ]:
args = {'import_on': True} if IMPORT else {}
l = Lattice2D(side1=side, geo=geo, pflip=pflip, **args)
ising_dyn = IsingDynamics(sg=l, T=T_ising, 
                          nstepsIsing=1000, 
                          ic="ground_state_0",
                          runlang="C0")
if not l.import_on:
    l.flip_random_fract_edges()
    l.export_graph()
    l.export_adj_bin()

In [ ]:
energies = []
for i in tqdm(range(navg)):
    ising_dyn.init_ising_dynamics()
    ising_dyn.run(eqSTEP=eqSTEP)
    sfxstr = ising_dyn.id_string_isingdyn if ising_dyn.id_string_isingdyn else "\"\""
    enepath = f"{l.isingpath}ene_{l.stdFname}_T={ising_dyn.T:.3g}_{sfxstr}.bin"
    ene = np.fromfile(enepath, dtype=float)
    energies.append(ene)

### C++ attempt (slow)

In [ ]:
temperature = 1
pflip = 0
side = 50
pbc=True
geo = 'squared'
navg = 5
steps = side**2
frameRate = 1
single_mode = True

In [ ]:
l = Lattice2D(side, geo=geo, pflip=pflip, pbc=pbc)
#
l.flip_random_fract_edges()
edgeList = list(l.GraphReprDict['H'].edges())
nodeList = l.nodeList
signs = list(l.GraphReprDict['H'].edges(data='weight'))

In [ ]:
if single_mode:
#
    model = ising_model_store.IsingModel(edgeList, signs, side, side, 
                                            temperature, "asynchronous")
    model.simulate(steps, frameRate)
    ene = np.array(model.getFrameEnergies())
else: 
    magn = []
    ene = []
    for _ in range(navg):
        model = ising_model_store.IsingModel(edgeList, signs, side, side, 
                                            temperature, "asynchronous")
        model.simulate(steps, frameRate)
        magn.append([model.getFrameMagnetizations()])
        ene.append([model.getFrameEnergies()])

In [ ]:
for en in ene:
    tt = en[0]
    eneNorm = tt
    plt.plot(np.linspace(1, len(eneNorm), num=len(eneNorm)), eneNorm, '-')
# plt.plot(magn, 'h')
plt.xscale('log')
